# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below.

## Import libraries

In [19]:
import pandas as pd
import numpy as np

#json tools
import json
from pandas.io.json import json_normalize

#scraping
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

#geocoders
from geopy.geocoders import Nominatim

#visualization libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

#kmeans clustering
from sklearn.cluster import KMeans

print('Done!')

Done!


## Task 1 - Scraping Wikipedia page, creating Pandas DF, cleaning data

Using BeautifulSoup and URLopen libraries

In [6]:
wlink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_page = urlopen(wlink).read().decode('utf-8')
page = BeautifulSoup(raw_page, 'html.parser')
table = page.body.table.tbody

Next, transforming the table data to Pandas Dataframe

In [7]:
#functions for getting cell and row data

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [8]:
#writing into pandas dataframe
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleaning the data:

1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
2. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed    twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with    a comma as shown in row 11 in the above table.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table      on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
4. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
5. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
#dropping the "Not Assigned" borough
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [10]:
#Consolidating the neighbourhoods that share the postcode

df_postcodes = df1['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
    
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
#Checking dataframe shape
df2.shape

(103, 3)

## Task 2 - Get Coordinates

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Using the provided Geospatial_Coordinates.csv file to get the coordinates:

In [12]:
#reading the file to coord dataframe
df2['Latitude'] = '0';
df2['Longitude'] = '0';

coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [13]:
#merging dataframe that contain coordinates with the one that contains borough names
for i in df2.index:
    for j in coord.index:
        if df2.iloc[i, 0] == coord.iloc[j, 0]:
            df2.iloc[i, 3] = coord.iloc[j, 1]
            df2.iloc[i, 4] = coord.iloc[j, 2]

#checking the results            
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


## Task 3 - Analysis

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

### 3.1 Select only the neighbourhoods of Downtown Toronto

Choose the neighbourhoods that contain word " Downtown Toronto":

In [15]:
toronto = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789


In [16]:
#get the coordinates for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [ ]:
#create the Folium map of Downtown Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2 Utilizing the Foursquare API to get top 100 venues in Downtown Toronto

In [20]:
#set credintials
CLIENT_ID = 'IPTYUZQHVW5OCDTT331BXA1SFQCJ3QCNQ2NVFZHQI5M4ZJLY' # your Foursquare ID
CLIENT_SECRET = '4ARF5SHATZIHJ2FJURJBFIUBZWYKR0UZ4FP5XHAGRE4BCZJ1' # your Foursquare Secret
VERSION = '20190323' # Foursquare API version

Borrowing the function from the lab to get Top 100 venues in Downtown Toronto within a radius of 500m:

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

downtown_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [23]:
#checking the size of venues dataframe
downtown_venues.shape

(1273, 7)

In [24]:
#checking how many unique categories of venues are there
print('There are {} unique categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 207 unique categories.


### 3.3 Analyze each neighbourhood

In [25]:
# one hot encoding
dt_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#checking the dataframe size
dt_onehot.shape

(1273, 207)

Gouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [27]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.00,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.012821,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010204,0.00,0.00,0.051020,0.000000,0.051020,0.010204,0.000000,0.000000,0.010204
6,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.012195,0.00,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.012195,0.012195,0.000000,0.012195,0.012195,0.000000
8,"Commerce Court,Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000


In [28]:
#checking the grouped dataframe size
dt_grouped.shape

(18, 207)

Printing out each neighborhood along with the top 5 most common venues in it:

In [29]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4             Café  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.05
2        Cocktail Bar  0.05
3  Italian Restaurant  0.03
4          Steakhouse  0.03


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.15
1  Airport Terminal  0.15
2   Airport Service  0.15
3     Boat or Ferry  0.08
4   Harbor / Marina  0.08


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2       Market  0.05
3         Café  0.05
4       Bakery  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4                 Bar  0.04




Converting the results to Pandas dataframe:

In [30]:
#function to sort the venues in descending order:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
#create the new dataframe and display the top 10 venues for each neighborhood:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Bar,Thai Restaurant,Café,American Restaurant,Hotel,Asian Restaurant,Sushi Restaurant,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Pub,Farmers Market,Bakery,Seafood Restaurant,Steakhouse,Cheese Shop
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport,Airport Food Court
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Bakery,Café,Market,Pizza Place,Pet Store
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sushi Restaurant,Bubble Tea Shop,Spa,Chinese Restaurant,Salad Place


### 3.4 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 4

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

dt_merged = toronto

# merge downtown_grouped with toronto data to add latitude/longitude for each neighborhood
# I realized that I've misspelled the NeighboUrhood column name in Toronto dataframe. oops...
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

dt_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775,1,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677,0,Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Bakery,Café,Market,Pizza Place,Pet Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,0,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Men's Store,Café,Mediterranean Restaurant
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Mexican Restaurant,Restaurant,Health Food Store
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Tea Room,Lingerie Store,Bubble Tea Shop,Pizza Place


Visualize the resulting clusters on map:

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.5 Examine the clusters

In [36]:
#Cluster 1
dt_merged.loc[dt_merged['Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Bakery,Café,Market,Pizza Place,Pet Store
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Men's Store,Café,Mediterranean Restaurant
3,Downtown Toronto,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Mexican Restaurant,Restaurant,Health Food Store
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Tea Room,Lingerie Store,Bubble Tea Shop,Pizza Place
5,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Bakery,Cosmetics Shop,Gastropub,Gym
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Pub,Farmers Market,Bakery,Seafood Restaurant,Steakhouse,Cheese Shop
7,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sushi Restaurant,Bubble Tea Shop,Spa,Chinese Restaurant,Salad Place
8,Downtown Toronto,0,Coffee Shop,Steakhouse,Bar,Thai Restaurant,Café,American Restaurant,Hotel,Asian Restaurant,Sushi Restaurant,Bakery
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Fried Chicken Joint,Brewery,Scenic Lookout,Pizza Place,Music Venue
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Gastropub,Bakery


In [37]:
#Cluster 2
dt_merged.loc[dt_merged['Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [38]:
#Cluster 3
dt_merged.loc[dt_merged['Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport,Airport Food Court


In [39]:
#Cluster 4
dt_merged.loc[dt_merged['Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,3,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Convenience Store,Nightclub,Diner,Restaurant,Baby Store


### 3.6 Conclusion:


As seen from the above dataframes corresponding to each cluster label, the following conclusions can be made:

1. Cluster 1: the most common venue type is Coffee Shop, followed by restaurants and bars.
2. Cluster 2: the most common venue type is Park or Playground.
3. Cluster 3: the most common venue type is Airport Lounge.
4. Cluster 4: the most common venue type is Grocery Store.
The most popular venue type in Downtown Toronto is Coffee Shop, containing 16 venues total.